In [2]:
import os
import re
import pickle
import pandas as pd
from bs4 import BeautifulSoup as bs

In [3]:
with open('df_page', 'rb') as f:
    df_page = pickle.load(f)
    
# display(df_institute)

In [4]:
df_true = df_page.query('table == True').reset_index(drop=True)
df_false = df_page.query('table == False').reset_index(drop=True)

# display(df_true)
# display(df_false)

In [5]:
df_new = (df_true
    .sort_values(['institute_id', 'term_id', 'school_id', 'updated'])
    .groupby(['institute_id', 'term_id', 'school_id'])
    .tail(1)
    .reset_index(drop=True))

df_old = (df_true
    .sort_values(['institute_id', 'term_id', 'school_id', 'updated'])
    .groupby(['institute_id', 'term_id', 'school_id'])
    .head(-1)
    .reset_index(drop=True))

display(df_new)
display(df_old)

,path,institute_id,term_id,school_id,table,updated
0,./2023-06-06/i=HAW&t=202130&s=ABRP,HAW,202130,ABRP,True,2023-06-06
1,./2023-06-06/i=HAW&t=202130&s=ACC,HAW,202130,ACC,True,2023-06-06
2,./2023-06-06/i=HAW&t=202130&s=AEC,HAW,202130,AEC,True,2023-06-06
3,./2023-06-06/i=HAW&t=202130&s=AG,HAW,202130,AG,True,2023-06-06
4,./2023-06-06/i=HAW&t=202130&s=AJ,HAW,202130,AJ,True,2023-06-06
...,...,...,...,...,...,...
9263,./2023-06-06/i=WOA&t=202410&s=SD,WOA,202410,SD,True,2023-06-06
9264,./2023-06-06/i=WOA&t=202410&s=SOC,WOA,202410,SOC,True,2023-06-06
9265,./2023-06-06/i=WOA&t=202410&s=SP,WOA,202410,SP,True,2023-06-06
9266,./2023-06-06/i=WOA&t=202410&s=SPED,WOA,202410,SPED,True,2023-06-06


,path,institute_id,term_id,school_id,table,updated
0,./2023-06-06/i=MAN&t=202340&s=JPN,MAN,202340,JPN,True,2023-06-06
1,./2023-06-06/i=MAN&t=202340&s=MATH,MAN,202340,MATH,True,2023-06-06


In [6]:
new_dicts = df_new.to_dict(orient='records')

row_dicts = []
for i, new_dict in enumerate(new_dicts):
    path = new_dict['path']
    with open(path, 'r') as f:
        text = f.read()
    html = bs(text, 'lxml')
    table = html.find('table', attrs={'class': 'listOfClasses'})
    ths = table.tr.find_all('th')
    trs = table.find_all('tr')[1:]
    for tr in trs:
        tds = tr.find_all('td')
        row_dict = {'path': path}
        for th, td in zip(ths, tds):
            row_dict[th.text] = td.text
        row_dicts.append(row_dict)
    print(f'\r{i + 1:<10}', end='')
print()

9268      


In [7]:
pattern = re.compile(r'^\d+$')
is_crn = lambda x: pattern.match(str(x)) is not None

df_row = pd.DataFrame(row_dicts)
df_row = df_row[df_row['CRN'].map(is_crn)]
df_row = df_row.reset_index(drop=True)
df_row = df_row[['path', 'CRN', 'Course', 'Title', 'Curr.Enrolled']]
df_row = df_row.rename(columns={
    'CRN': 'class_id',
    'Course': 'course',
    'Title': 'title',
    'Curr.Enrolled': 'enrolled'})
df_row = df_row.astype({'class_id': 'int64', 'enrolled': 'int64'})

display(df_row)

,path,class_id,course,title,enrolled
0,./2023-06-06/i=HAW&t=202130&s=ABRP,18356,ABRP 50A,Frame Measure & Alignment Tech,10
1,./2023-06-06/i=HAW&t=202130&s=ABRP,18357,ABRP 120,Metal and Plastic Refinishing,8
2,./2023-06-06/i=HAW&t=202130&s=ACC,18281,ACC 120,College Accounting I,28
3,./2023-06-06/i=HAW&t=202130&s=ACC,18282,ACC 124,Principles of Accounting I,14
4,./2023-06-06/i=HAW&t=202130&s=ACC,18283,ACC 134,Individual Income Tax Prep,20
...,...,...,...,...,...
84815,./2023-06-06/i=WOA&t=202410&s=SSCI,68273,SSCI 486S,Senior Project Sociology,5
84816,./2023-06-06/i=WOA&t=202410&s=SSCI,68267,SSCI 490A,Sr Practicum-Anthropology,8
84817,./2023-06-06/i=WOA&t=202410&s=SSCI,68248,SSCI 490C,SSCI Pract-EarlyChldhd Ed,7
84818,./2023-06-06/i=WOA&t=202410&s=SSCI,68268,SSCI 490F,Social Sciences Practicum,0


In [8]:
df_class = df_new.merge(df_row)[[
    'institute_id',
    'term_id',
    'class_id',
    'school_id',
    'course',
    'title',
    'enrolled',
    'updated']]

display(df_class)

,institute_id,term_id,class_id,school_id,course,title,enrolled,updated
0,HAW,202130,18356,ABRP,ABRP 50A,Frame Measure & Alignment Tech,10,2023-06-06
1,HAW,202130,18357,ABRP,ABRP 120,Metal and Plastic Refinishing,8,2023-06-06
2,HAW,202130,18281,ACC,ACC 120,College Accounting I,28,2023-06-06
3,HAW,202130,18282,ACC,ACC 124,Principles of Accounting I,14,2023-06-06
4,HAW,202130,18283,ACC,ACC 134,Individual Income Tax Prep,20,2023-06-06
...,...,...,...,...,...,...,...,...
84815,WOA,202410,68273,SSCI,SSCI 486S,Senior Project Sociology,5,2023-06-06
84816,WOA,202410,68267,SSCI,SSCI 490A,Sr Practicum-Anthropology,8,2023-06-06
84817,WOA,202410,68248,SSCI,SSCI 490C,SSCI Pract-EarlyChldhd Ed,7,2023-06-06
84818,WOA,202410,68268,SSCI,SSCI 490F,Social Sciences Practicum,0,2023-06-06


In [9]:
with open('df_class', 'wb') as f:
    pickle.dump(df_class, f)